In [34]:
# Python
import pandas as pd
import numpy as np
#
from pathlib import Path
from datetime import datetime
#
import xgboost
#
import importlib
import utilities.train_test as train_test
import utilities.comparison as comparison

In [35]:
df = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')
df_overview = pd.read_csv('../../../data/df_overview.csv', index_col=0)

In [36]:
file = Path("../../../data/df_tabular.csv")

if file.exists():
    df_tabular = pd.read_csv(file, index_col=0)
else:
    importlib.reload(train_test)
    df_tabular = train_test.get_dataframe_tabular(df)
    df_tabular.to_csv(file)

In [37]:
df_tabular

,month,year,date,m_return(t-11),m_return(t-10),m_return(t-9),m_return(t-8),m_return(t-7),m_return(t-6),m_return(t-5),...,m_return_target(t+4),m_return_target(t+5),m_return_target(t+6),m_return_target(t+7),m_return_target(t+8),m_return_target(t+9),m_return_target(t+10),m_return_target(t+11),m_return_target(t+12),stock_ticker_label
0,10,2000,2000-10-01,1.13,1.11,1.02,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.01,1.0,1.00,1.00,1371
1,11,2000,2000-11-01,1.11,1.02,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.01,1.00,1.0,1.00,1.00,1371
2,12,2000,2000-12-01,1.02,1.00,1.00,1.00,1.00,1.00,1.01,...,1.00,1.00,1.00,1.01,1.00,1.00,1.0,1.00,1.00,1371
3,1,2001,2001-01-01,1.00,1.00,1.00,1.00,1.00,1.01,1.00,...,1.00,1.00,1.01,1.00,1.00,1.00,1.0,1.00,1.01,1371
4,2,2001,2001-02-01,1.00,1.00,1.00,1.00,1.01,1.00,1.00,...,1.00,1.01,1.00,1.00,1.00,1.00,1.0,1.01,1.00,1371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474406,4,2024,2024-04-01,0.98,1.07,1.13,0.95,0.93,1.02,1.09,...,1.04,0.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1549
474407,5,2024,2024-05-01,1.07,1.13,0.95,0.93,1.02,1.09,1.09,...,0.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1549
474408,6,2024,2024-06-01,1.13,0.95,0.93,1.02,1.09,1.09,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1549
474409,7,2024,2024-07-01,0.95,0.93,1.02,1.09,1.09,1.00,1.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1549


### Missing values

In [38]:
df_tabular[df_tabular["m_return_target(t+1)"].isna()][["date", "stock_ticker_label"]]

,date,stock_ticker_label


## Direct forecasting

## 1 Month

In [41]:
importlib.reload(train_test)

X_train, y_train, X_test, y_test, min_datestr = train_test.split_train_test_tabular(df_tabular, months=60)

model_1m = xgboost.XGBRegressor(n_estimators=1000)
model_1m.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [43]:
importlib.reload(train_test)
months_1m = 1
X_train_1m, y_train_1m, X_test_1m, y_test_1m, min_datestr = train_test.split_train_test_tabular(df_tabular, months=1)
# reset index
X_train_1m.reset_index(drop=True, inplace=True)
y_train_1m.reset_index(drop=True, inplace=True)
X_test_1m.reset_index(drop=True, inplace=True)
y_test_1m.reset_index(drop=True, inplace=True)

In [44]:
dt = datetime.strptime(min_datestr, '%Y-%m-%d')

# get all training months without the last month
X_train_input_1m = X_train_1m.head(len(X_train_1m) - len(df.columns) * months_1m)
# Get last month of the training dataset and use as input to predict the next month
X_test_input_1m = X_train_1m.tail(len(df.columns) * months_1m)

# Trained 
y_train_pred_1m = model_1m.predict(X_train_input_1m)
# Predictions
y_test_pred_1m = model_1m.predict(X_test_input_1m)

### 1 month Actual vs Prediction

In [46]:
importlib.reload(comparison)
#
y_train_mean_pred_1m, y_test_mean_pred_1m = comparison.get_train_test_mean_pred(y_train_pred_1m, y_test_pred_1m, len(df.columns))
#
comparison.generate_plot(df, df_tabular, y_train_mean_pred_1m, y_test_mean_pred_1m)

286 285 1


#### Overview table

In [47]:
importlib.reload(comparison)

# Get train true values followed with predicted month/s
y_train_1m_list = y_train_1m['m_return_target(t+1)'].tolist()
y_test_1m_list = y_test_pred_1m.tolist()
#
df_to_evaluate_1m = comparison.get_df_from_pred_list(df, y_train_1m_list, y_test_1m_list)
df_to_evaluate_1m = df_to_evaluate_1m - 1
df_to_evaluate_1m

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.000000,0.00000,-0.100000,0.000000,-0.230000,0.010000,-0.340000,0.000000,0.000000,-0.060000,...,0.000000,0.00000,0.000000,0.070000,0.00000,-0.340000,0.000000,0.00000,0.000000,-0.010000
1,0.000000,0.00000,0.000000,-0.060000,0.030000,0.050000,0.220000,0.000000,-0.040000,0.050000,...,0.000000,0.00000,0.000000,0.090000,0.00000,-0.160000,0.000000,0.00000,0.000000,0.090000
2,0.010000,0.00000,0.550000,0.040000,0.090000,-0.020000,0.280000,-0.080000,0.020000,-0.060000,...,0.000000,0.29000,-0.090000,-0.020000,-0.04000,0.690000,-0.140000,-0.24000,0.000000,0.050000
3,0.000000,-0.03000,0.090000,-0.130000,-0.050000,-0.010000,-0.110000,-0.020000,-0.100000,-0.080000,...,-0.480000,0.00000,0.000000,0.010000,0.00000,-0.260000,0.000000,0.00000,0.000000,0.040000
4,0.000000,0.00000,-0.060000,0.000000,-0.150000,-0.070000,-0.160000,0.000000,0.000000,0.020000,...,1.180000,0.00000,0.000000,0.050000,0.00000,0.150000,0.000000,0.00000,0.000000,0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,-0.050000,0.10000,0.060000,0.020000,0.140000,0.120000,0.120000,0.060000,0.060000,0.060000,...,0.090000,0.07000,0.160000,0.050000,-0.01000,0.080000,0.030000,-0.02000,-0.140000,-0.020000
283,0.000000,-0.04000,-0.040000,-0.030000,-0.050000,-0.040000,-0.060000,-0.060000,-0.050000,0.110000,...,0.070000,-0.09000,0.180000,-0.110000,0.01000,-0.100000,0.190000,0.01000,0.140000,-0.020000
284,0.190000,0.08000,0.020000,0.060000,0.120000,0.220000,0.050000,0.090000,0.070000,-0.080000,...,0.100000,0.07000,0.060000,0.160000,-0.06000,-0.040000,0.140000,0.28000,-0.100000,0.090000
285,-0.040000,-0.22000,0.060000,0.080000,-0.020000,-0.020000,0.030000,0.070000,0.100000,0.060000,...,-0.050000,0.04000,0.160000,-0.130000,-0.04000,0.000000,-0.030000,-0.07000,-0.110000,0.040000


In [48]:
importlib.reload(comparison)
df_pred_1m, raw_weights_1m, mu_1m, S_1m, sigma_1m, sharpe_1m = comparison.get_portfolio_performance(df_to_evaluate_1m,
                                                                                                    "xgbooster_weights_1m.csv",
                                                                                                    min_avg_return=-0.5,
                                                                                                    months=12)

allocation, leftover = comparison.create_discrete_allocation(df_pred_1m, raw_weights_1m)

# Create overview
df_view_1m = pd.DataFrame.from_dict(raw_weights_1m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_1m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_1m)), index=S_1m.columns).values
# Set annual returns
df_view_1m['avg_annual_return'] = mu_1m.values
#
df_view_1m['return_last_period(1m)'] = round(df.tail(1).prod() - 1, 2)
#
df_view_1m

Expected annual return: 65.1%
Annual volatility: 9.4%
Sharpe Ratio: 6.73
Discrete allocation: {'RS1.L': 96, 'KE': 82, 'TEG.DE': 2145, 'SCS': 206, 'ACCO': 225, 'VNA.DE': 68, '7912.T': 547, 'KEYS': 15850, 'SGRO.L': 395, 'CBRE': 314, 'PGRE': 487, 'BBOX.L': 205, 'PLXS': 19417, 'LAND.L': 67, 'OLED': 24, 'CDW': 50, 'FLEX': 717, 'REZI': 359, 'CMPR': 20, 'DLAR.L': 390, '2379.T': 32, '6754.T': 34, 'SHA.DE': 95, 'TTMI': 75, '3234.T': 84, 'ASGN': 104, 'COA.L': 224, 'VVV': 28, 'PAGE.L': 41, '3309.T': 2538, 'STWD': 1114, 'GROW.L': 54, 'KFY': 51, 'MEI': 36, 'RGLD': 66, 'IWG.L': 1, 'ZIL2.DE': 258, 'SANM': 72, 'ACN': 33, '7893.T': 106, 'AGNC': 268, 'CTS': 36, 'SCSC': 308, '3283.T': 62, 'PDM': 134, 'III.L': 665, 'NBS.L': 47, 'DSCV.L': 29, '6183.T': 55, 'KELYA': 135, '4384.T': 41, 'BRBY.L': 23, '7984.T': 671, 'CCK': 37, 'APTV': 35, 'STEM.L': 17618, 'UU.L': 9493, 'ITRI': 23623, 'INF.L': 61, '4218.T': 217, 'HMSO.L': 153, 'WDC': 189, '7313.T': 283, 'EBF': 125, 'TNET': 36, '6098.T': 18127, 'HLMA.L': 39, 'WP

,max_sharpe_weight,avg_annual_volatility,avg_annual_return,return_last_period(1m)
RS1.L,0.00000,1.576157,0.113799,-0.01
KE,0.00000,1.631618,-0.301001,-0.05
TEG.DE,0.00235,1.604974,0.498377,0.10
LEG.DE,0.00000,1.672290,0.364967,0.08
SCS,0.00000,1.595129,0.321200,-0.07
...,...,...,...,...
KVHI,0.00000,1.510988,-0.079269,0.00
MOON.L,0.00000,1.586560,0.257133,-0.05
NEO,0.00000,1.937570,0.357876,-0.05
6055.T,0.00000,1.737581,-0.214965,-0.06


## 6 Months

In [83]:
importlib.reload(train_test)
X_train, y_train, X_test, y_test, min_datestr = train_test.split_train_test_tabular(df_tabular, months=60, target_key='m_return_target(t+6)')

# 
model_6m = xgboost.XGBRegressor(n_estimators=1000)
model_6m.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=False)

print(len(X_train)/1653, len(X_test)/1653)

227.0 55.0


In [50]:
months_6m = 6
X_train_6m, y_train_6m, X_test_6m, y_test_6m, min_datestr = train_test.split_train_test_tabular(df_tabular, months=6,
                                                                                              target_key='m_return_target(t+6)')

In [51]:
dt = datetime.strptime(min_datestr, '%Y-%m-%d')

# get all training months without the last 6 months
X_train_input_6m = X_train_6m.head(len(X_train_6m) - len(df.columns)*months_6m)
# Get last 6 months of the training dataset and use as input to predict the next 6 months
X_test_input_6m = X_train_6m.tail(len(df.columns)*months_6m)

# Trained 
y_train_pred_6m = model_6m.predict(X_train_input_6m)
# Predictions
y_test_pred_6m = model_6m.predict(X_test_input_6m)

### 6 Months Actual vs Prediction

In [52]:
importlib.reload(comparison)
#
y_train_mean_pred_6m, y_test_mean_pred_6m = comparison.get_train_test_mean_pred(y_train_pred_6m, y_test_pred_6m, len(df.columns))
#
comparison.generate_plot(df, df_tabular, y_train_mean_pred_6m, y_test_mean_pred_6m)

281 275 6


#### Overview table

In [53]:
# Get train true values followed with predicted month/s
y_train_6m_list = y_train_6m['m_return_target(t+6)'].tolist()
y_pred_6m_list = y_test_pred_6m.tolist()
#
df_to_evaluate_6m = comparison.get_df_from_pred_list(df, y_train_1m_list, y_pred_6m_list)
df_to_evaluate_6m = df_to_evaluate_6m - 1
df_to_evaluate_6m

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.000000,0.000000,-0.100000,0.000000,-0.230000,0.010000,-0.340000,0.000000,0.000000,-0.060000,...,0.000000,0.000000,0.000000,0.070000,0.000000,-0.340000,0.000000,0.000000,0.000000,-0.010000
1,0.000000,0.000000,0.000000,-0.060000,0.030000,0.050000,0.220000,0.000000,-0.040000,0.050000,...,0.000000,0.000000,0.000000,0.090000,0.000000,-0.160000,0.000000,0.000000,0.000000,0.090000
2,0.010000,0.000000,0.550000,0.040000,0.090000,-0.020000,0.280000,-0.080000,0.020000,-0.060000,...,0.000000,0.290000,-0.090000,-0.020000,-0.040000,0.690000,-0.140000,-0.240000,0.000000,0.050000
3,0.000000,-0.030000,0.090000,-0.130000,-0.050000,-0.010000,-0.110000,-0.020000,-0.100000,-0.080000,...,-0.480000,0.000000,0.000000,0.010000,0.000000,-0.260000,0.000000,0.000000,0.000000,0.040000
4,0.000000,0.000000,-0.060000,0.000000,-0.150000,-0.070000,-0.160000,0.000000,0.000000,0.020000,...,1.180000,0.000000,0.000000,0.050000,0.000000,0.150000,0.000000,0.000000,0.000000,0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,0.123534,0.070807,-0.058092,-0.253677,0.117354,0.092943,0.331190,0.321947,0.026515,-0.168888,...,0.012532,-0.026865,-0.022313,-0.001150,0.021679,0.032163,-0.031457,0.037416,0.011705,-0.019574
288,-0.001414,-0.068942,-0.198856,0.178871,0.215746,-0.193470,-0.092314,-0.067086,0.110363,0.077282,...,0.032217,-0.080730,-0.005133,-0.175693,0.041845,-0.109208,0.010885,-0.159985,0.034830,-0.155161
289,-0.008178,-0.103794,0.212474,-0.202204,-0.210486,0.162061,0.122468,-0.108271,-0.025298,-0.009303,...,0.033886,-0.007528,0.009432,-0.015825,0.021895,-0.073570,-0.006834,-0.064793,0.005849,-0.025031
290,-0.011283,-0.252874,0.054542,-0.259109,-0.204893,0.040330,0.199532,-0.086460,-0.138622,0.158513,...,-0.001989,-0.017072,0.071985,0.010393,0.007745,0.000075,-0.068331,-0.031193,-0.016525,-0.009328


In [54]:
importlib.reload(comparison)
df_pred_6m, raw_weights_6m, mu_6m, S_6m, sigma_6m, sharpe_6m = comparison.get_portfolio_performance(df_to_evaluate_6m,
                                                                                                    "xgbooster_weights_6m.csv",
                                                                                                    min_avg_return=-0.5,
                                                                                                    months=12)
comparison.create_discrete_allocation(df_pred_6m, raw_weights_6m)

# Create overview
df_view_6m = pd.DataFrame.from_dict(raw_weights_6m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_6m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_6m)), index=S_6m.columns).values
# Set annual returns
df_view_6m['avg_annual_return'] = mu_6m.values
#
df_view_6m['return_last_period(6m)'] = round(df.tail(6).prod() - 1, 2)
df_view_6m

Expected annual return: 90.1%
Annual volatility: 11.1%
Sharpe Ratio: 7.91
Discrete allocation: {'PGRE': 25, 'PSON.L': 31, 'HAS': 899, 'TPL': 5905, 'ARW': 95, 'RYN': 74549, 'CMPR': 166, 'DLAR.L': 787, '2379.T': 28259, '6754.T': 83, 'PBI': 244, 'BYG.L': 772, 'COA.L': 677, 'STWD': 24, 'MTO.L': 139, 'IWG.L': 853, '8954.T': 26044, 'SVS.L': 252, 'FUTR.L': 1733, 'III.L': 1557, '6724.T': 30169, 'BRBY.L': 28008, '7984.T': 3929, 'WDC': 55, 'HLMA.L': 1312, 'WPP.L': 70, '3279.T': 66, 'JEN.DE': 387, 'LXP': 318, 'WKP.L': 247, '7951.T': 1693, 'KRG': 278, 'ATR': 30, 'ELV': 766, 'NYT': 232, 'SNWS.L': 1277, 'INCH.L': 231, 'NWSA': 20, 'EXPN.L': 109, 'CUZ': 394, '8151.T': 3429, 'RHI': 29386, '7868.T': 11065, 'LSL.L': 4037, 'ATGE': 956, '7701.T': 1255, 'GWI.L': 145, 'ZWS': 73888, 'VRE': 194, 'KMX': 48061, 'OXIG.L': 75575, 'REG': 241, '7976.T': 127, 'STAA': 1531, 'PNN.L': 108, 'ENR.DE': 3351, 'WSM': 3341, '8952.T': 613, 'PTEC.L': 2752, '8976.T': 205, 'STX': 67, 'ANIK': 51682, '4848.T': 72, '5108.T': 82398, 

,max_sharpe_weight,avg_annual_volatility,avg_annual_return,return_last_period(6m)
RS1.L,0.00000,1.769312,0.134183,0.09
KE,0.00000,1.862518,-0.497960,-0.18
TEG.DE,0.00166,1.868049,0.047628,0.28
LEG.DE,0.00000,2.001607,-0.213589,0.22
SCS,0.00000,1.981646,-0.324061,0.02
...,...,...,...,...
KVHI,0.00000,1.751109,-0.224551,-0.12
MOON.L,0.00000,1.795452,0.041699,0.18
NEO,0.00000,1.862586,-0.228996,-0.00
6055.T,0.00000,1.787631,-0.305866,-0.31


## 12 Months 

In [55]:
importlib.reload(train_test)
X_train, y_train, X_test, y_test, min_datestr = train_test.split_train_test_tabular(df_tabular, months=60, target_key='m_return_target(t+12)')

# 
model_12m = xgboost.XGBRegressor(n_estimators=1000)
model_12m.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [73]:
print(len(X_train)/1653, len(X_test)/1653)

227.0 49.0


In [56]:
months_12m = 12
X_train_12m, y_train_12m, X_test_12m, y_test_12m, min_datestr = train_test.split_train_test_tabular(df_tabular, months=months_12m,
                                                                                                  target_key='m_return_target(t+12)')

In [57]:
dt = datetime.strptime(min_datestr, '%Y-%m-%d')

# get all training months without the last 12 months
X_train_input_12m = X_train_12m.head(len(X_train_12m) - len(df.columns)*months_12m)
# Get last 6 months of the training dataset and use as input to predict the next 12 months
X_test_input_12m = X_train_12m.tail(len(df.columns)*months_12m)

# Trained 
y_train_pred_12m = model_12m.predict(X_train_input_12m)
# Predictions
y_test_pred_12m = model_6m.predict(X_test_input_12m)

### 12 Months Actual vs Prediction

In [58]:
importlib.reload(comparison)
#
importlib.reload(comparison)
y_train_mean_pred_12m, y_test_mean_pred_12m = comparison.get_train_test_mean_pred(y_train_pred_12m, y_test_pred_12m, len(df.columns))
#
comparison.generate_plot(df, df_tabular, y_train_mean_pred_12m, y_test_mean_pred_12m)

#### Overview table

In [59]:
importlib.reload(comparison)

# Get train true values followed with predicted month/s
y_train_12m_list = y_train_12m['m_return_target(t+12)'].tolist()
y_test_12m_list = y_test_pred_12m.tolist()
#
df_to_evaluate_12m = comparison.get_df_from_pred_list(df, y_train_12m_list, y_test_12m_list)
df_to_evaluate_12m = df_to_evaluate_12m - 1
df_to_evaluate_12m

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.000000,0.000000,0.320000,0.000000,0.050000,0.100000,0.140000,0.000000,0.000000,0.140000,...,0.000000,0.000000,0.000000,0.040000,0.000000,0.530000,0.000000,0.000000,0.010000,0.110000
1,0.000000,0.000000,0.050000,-0.020000,0.090000,0.060000,0.150000,1.430000,-0.020000,0.010000,...,0.000000,0.000000,0.000000,0.050000,0.000000,-0.170000,0.000000,0.000000,0.000000,0.060000
2,0.000000,0.000000,-0.150000,0.000000,0.040000,0.090000,0.070000,0.000000,0.000000,-0.010000,...,-0.120000,0.000000,0.000000,0.020000,0.000000,-0.010000,0.000000,0.000000,0.000000,0.040000
3,0.010000,0.000000,0.040000,0.000000,0.070000,0.010000,0.050000,-0.650000,0.000000,-0.100000,...,0.000000,0.000000,0.000000,0.010000,0.000000,0.230000,0.000000,-0.050000,-0.010000,0.090000
4,0.000000,0.000000,-0.020000,0.000000,-0.030000,-0.030000,-0.010000,0.060000,0.000000,0.100000,...,0.010000,-0.100000,0.000000,-0.030000,0.000000,0.020000,0.000000,0.000000,0.010000,-0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,0.000292,-0.017971,-0.019445,0.069847,0.085843,-0.024792,0.043555,0.036488,0.017755,-0.050697,...,0.017791,0.034984,-0.009036,0.008564,-0.060288,0.012532,-0.022313,0.021679,-0.031457,0.011705
283,0.025323,0.054896,0.008362,0.056536,-0.056524,0.043779,0.048160,0.050901,0.019647,-0.019775,...,0.026269,-0.001177,0.052536,0.043730,-0.014902,0.032217,-0.005133,0.041845,0.010885,0.034830
284,0.013769,0.068290,0.001476,-0.028086,-0.066652,0.052008,-0.019236,0.026775,0.028045,-0.053575,...,0.019490,-0.016612,0.011353,-0.005469,-0.095797,0.033886,0.009432,0.021895,-0.006834,0.005849
285,0.032251,0.002045,0.007750,-0.049634,-0.040268,0.023260,-0.014981,0.023231,0.016106,-0.022185,...,-0.004113,-0.005879,0.021267,0.032262,0.089933,-0.001989,0.071985,0.007745,-0.068331,-0.016525


In [60]:
importlib.reload(comparison)
df_pred_12m, raw_weights_12m, mu_12m, S_12m, sigma_12m, sharpe_12m = comparison.get_portfolio_performance(df_to_evaluate_12m,
                                                                                                          "xgbooster_weights_12m.csv",
                                                                                                          min_avg_return=-0.5,
                                                                                                          months=12)
comparison.create_discrete_allocation(df_pred_12m, raw_weights_12m)

# Create overview
df_view_12m = pd.DataFrame.from_dict(raw_weights_12m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_12m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_12m)), index=S_12m.columns).values
# Set annual returns
df_view_12m['avg_annual_return'] = mu_12m.values
# 
df_view_12m['return_last_period(12m)'] = round(df.tail(12).prod() - 1, 2)
df_view_12m

Expected annual return: 373.9%
Annual volatility: 16.7%
Sharpe Ratio: 22.25
Discrete allocation: {'KE': 436, 'LEG.DE': 113, 'SCS': 97, 'HNI': 1536, 'ACCO': 259, '7912.T': 449, 'KEYS': 22, 'SGRO.L': 98, 'BRC': 483, 'PGRE': 50, 'BBOX.L': 429, 'REL.L': 76, 'BHE': 228, 'PLXS': 687, 'HAS': 618, 'LAND.L': 374, 'OLED': 15355, 'CDW': 367, 'ARW': 127, 'FN': 989, 'NOVT': 4744, 'RYN': 7655, 'SNX': 21156, 'CMPR': 22316, 'DLAR.L': 433, 'SHA.DE': 422, 'TTMI': 428, '3234.T': 166, 'ASGN': 125, 'HOUS': 81, 'VVV': 229, 'PAGE.L': 55, 'DB1.DE': 625, 'AVB': 96, 'STWD': 220, 'GROW.L': 74, 'MTO.L': 2514, 'MEI': 248, 'RGLD': 283, 'KFRC': 212, 'IWG.L': 802, '8954.T': 541, 'ZIL2.DE': 2829, 'SANM': 119, 'CTS': 108, 'IPG': 43, 'BMI': 361, 'FUTR.L': 79, 'MAN': 154, '7994.T': 126, '3283.T': 61, 'PDM': 51, 'III.L': 300, 'NBS.L': 212, 'KELYA': 498, '4384.T': 1910, 'UU.L': 33, '6287.T': 249, 'HMSO.L': 148, 'WDC': 34, '7313.T': 135, 'TNET': 4787, '6098.T': 265, 'WPP.L': 131, 'JBL': 109, '3279.T': 312, 'JEN.DE': 112, 'T

,max_sharpe_weight,avg_annual_volatility,avg_annual_return,return_last_period(12m)
RS1.L,0.00029,1.718741,0.164886,0.08
KE,0.00020,1.734763,-0.034612,-0.35
TEG.DE,0.00000,1.759762,-0.241269,0.63
LEG.DE,0.00060,1.837286,0.357127,0.49
SCS,0.00000,1.778105,-0.426986,0.22
...,...,...,...,...
KVHI,0.00000,1.735725,-0.051170,-0.12
MOON.L,0.00000,1.726650,-0.075088,0.22
NEO,0.00000,1.754121,-0.157139,0.28
6055.T,0.00000,1.731777,-0.316597,-0.26
